# Part 1

In [1]:
INPUT_FILE = "day4.txt"

In [2]:
with open(INPUT_FILE) as f:
    loglines = [s.strip() for s in f.readlines()]

In [3]:
loglines[:5]

['[1518-03-31 00:26] falls asleep',
 '[1518-04-03 00:38] wakes up',
 '[1518-11-21 00:14] falls asleep',
 '[1518-07-21 00:34] wakes up',
 '[1518-04-02 00:20] falls asleep']

In [21]:
from collections import namedtuple, defaultdict
from datetime import datetime

In [26]:
ASLEEP = "falls asleep"
WAKE_UP = "wakes up"

In [24]:
Event = namedtuple("Event", "time event")
Interval = namedtuple("Interval", "start end activity")

In [27]:
def parse(s):
    time_str, event = s.split("]")
    time_str = time_str[1:]
    event = event.strip()
    return Event(datetime.strptime(time_str, "%Y-%m-%d %H:%M"), event)

def get_guard_id(s):
    _, gid, _, _ = s.split()
    return gid

In [28]:
events = list(sorted((parse(s) for s in loglines)))

In [29]:
events[:15]

[Event(time=datetime.datetime(1518, 3, 19, 0, 2), event='Guard #2953 begins shift'),
 Event(time=datetime.datetime(1518, 3, 19, 0, 37), event='falls asleep'),
 Event(time=datetime.datetime(1518, 3, 19, 0, 41), event='wakes up'),
 Event(time=datetime.datetime(1518, 3, 20, 0, 0), event='Guard #457 begins shift'),
 Event(time=datetime.datetime(1518, 3, 20, 0, 14), event='falls asleep'),
 Event(time=datetime.datetime(1518, 3, 20, 0, 26), event='wakes up'),
 Event(time=datetime.datetime(1518, 3, 21, 0, 3), event='Guard #691 begins shift'),
 Event(time=datetime.datetime(1518, 3, 21, 0, 31), event='falls asleep'),
 Event(time=datetime.datetime(1518, 3, 21, 0, 59), event='wakes up'),
 Event(time=datetime.datetime(1518, 3, 22, 0, 0), event='Guard #503 begins shift'),
 Event(time=datetime.datetime(1518, 3, 22, 0, 17), event='falls asleep'),
 Event(time=datetime.datetime(1518, 3, 22, 0, 36), event='wakes up'),
 Event(time=datetime.datetime(1518, 3, 23, 0, 2), event='Guard #223 begins shift'),
 Ev

In [30]:
guard_events = defaultdict(list)

In [32]:
current_guard = None
sleep_start = 0
prev_time = 0

for time, event in events:
    if event == ASLEEP:
        # Assuming guard doesn't sleep twice without waking, i.e. valid events
        interval = Interval(prev_time, time, "awake")
        guard_events[current_guard].append(interval)
    elif event == WAKE_UP:
        # Record current guard as asleep from last time until now
        interval = Interval(prev_time, time, "asleep")
        guard_events[current_guard].append(interval)
    else:
        # Record current guard as awake from last known time till now
        interval = Interval(prev_time, time, "awake")
        guard_events[current_guard].append(interval)
        
        # Change of guard
        current_guard = get_guard_id(event)
        
    prev_time = time

In [34]:
del guard_events[None]

In [36]:
def get_total_sleep_time(intervals):
    sleep_intervals = [i for i in intervals if i.activity == "asleep"]
    times = [(i.end - i.start).total_seconds() for i in sleep_intervals]
    return sum(times)

In [37]:
guard_total_sleep_times = {}
max_sleep = 0
max_sleep_guard = None
for guard, intervals in guard_events.items():
    this_guard_slept_for = get_total_sleep_time(intervals)
    if this_guard_slept_for > max_sleep:
        max_sleep = this_guard_slept_for
        max_sleep_guard = guard

In [40]:
max_sleep_guard, max_sleep

('#2953', 27900.0)

In [41]:
asleep_intervals_for_guard_2953 = [i for i in guard_events["#2953"] if i.activity == "asleep"]

In [45]:
sleep_times = [0]*60

In [46]:
for i in asleep_intervals_for_guard_2953:
    for j in range(i.start.minute, i.end.minute):
        sleep_times[j] += 1

In [54]:
max(enumerate(sleep_times), key=lambda k: k[1])

(39, 16)

In [55]:
39*2953

115167

# Part 2

In [56]:
len(guard_events.keys())

23

In [57]:
guard_sleeps = defaultdict(lambda: defaultdict(int))

In [58]:
for guard, intervals in guard_events.items():
    sleeps = [i for i in intervals if i.activity == "asleep"]
    for start, end, _ in sleeps:
        for k in range(start.minute, end.minute):
            guard_sleeps[guard][k] += 1

In [63]:
max_sleep_minute = -1
max_sleep_freq = 0
max_guard = None
for guard, sleep_mins in guard_sleeps.items():
    max_sleep_minute_guard, max_sleep_freq_guard = max(sleep_mins.items(), key=lambda p: p[1])
    if max_sleep_freq_guard > max_sleep_freq:
        max_sleep_freq = max_sleep_freq_guard
        max_sleep_minute = max_sleep_minute_guard
        max_guard = guard

In [66]:
max_guard, max_sleep_minute

('#1069', 30)

In [67]:
1069*30

32070